#### Question 1
In a practical, we saw the scs function (copied below along with overlap) for finding the shortest common superstring of a set of strings.

In [1]:
def overlap(a, b, min_length=3):
    """ Return length of longest suffix of 'a' matching
        a prefix of 'b' that is at least 'min_length'
        characters long.  If no such overlap exists,
        return 0. """
    start = 0  # start all the way at the left
    while True:
        start = a.find(b[:min_length], start)  # look for b's suffx in a
        if start == -1:  # no more occurrences to right
            return 0
        # found occurrence; check for full suffix/prefix match
        if b.startswith(a[start:]):
            return len(a)-start
        start += 1  # move just past previous match

import itertools

def scs(ss):
    """ Returns shortest common superstring of given
        strings, which must be the same length """
    shortest_sup = None
    for ssperm in itertools.permutations(ss):
        sup = ssperm[0]  # superstring starts as first string
        for i in range(len(ss)-1):
            # overlap adjacent strings A and B in the permutation
            olen = overlap(ssperm[i], ssperm[i+1], min_length=1)
            # add non-overlapping portion of B to superstring
            sup += ssperm[i+1][olen:]
        if shortest_sup is None or len(sup) < len(shortest_sup):
            shortest_sup = sup  # found shorter superstring
    return shortest_sup  # return shortest

It's possible for there to be multiple different shortest common superstrings for the same set of input strings. Consider the input strings:

- **ABC**
- **BCA**
- **CAB**

One shortest common superstring is **ABCAB**, but another is **BCABC**, and another is **CABCA**.

Now, what is the length of the shortest common superstring of the following strings?

- **CCT**
- **CTT**
- **TGC**
- **TGG**
- **GAT**
- **ATT**

In [2]:
strings = ['CCT', 'CTT', 'TGC','TGG','GAT','ATT']

In [3]:
len(scs(strings))

11

#### Question 2
How many different shortest common superstrings are there for the input strings given in the previous question?

Hint 1: You can modify the scs function to keep track of this. 

Hint 2: You can look at these examples to double-check that your modified scs is working as expected. 

In [4]:
from itertools import permutations

def scs_list(strings):
    """Return all shortest common superstrings from all permutations of input strings."""
    shortest_sup = None
    result = set()
    
    for ssperm in permutations(strings):
        sup = ssperm[0]
        for i in range(len(strings)-1):
            olen = overlap(sup, ssperm[i+1], min_length=1)
            sup += ssperm[i+1][olen:]
        
        if shortest_sup is None or len(sup) < len(shortest_sup):
            shortest_sup = sup
            result = {sup}
        elif len(sup) == len(shortest_sup):
            result.add(sup)
    
    return sorted(result)


In [5]:
len(scs_list(strings))

4

#### Question 3

All the reads are the same length (100 bases) and are exact copies of substrings from the forward strand of the virus genome.  You don't have to worry about sequencing errors, ploidy, or reads coming from the reverse strand.

Assemble these reads using one of the approaches discussed, such as greedy shortest common superstring.  Since there are many reads, you might consider ways to make the algorithm faster, such as the one discussed in the programming assignment in the previous module.

How many As (Adenines) are there in the full, assembled genome?

Hint: the virus genome you are assembling is exactly 15,894 bases long

In [8]:
def readFastq(filename):
    sequences = []
    qualities = []
    with open(filename) as fh:
        while True:
            fh.readline()  # skip name line
            seq = fh.readline().rstrip()  # read base sequence
            fh.readline()  # skip placeholder line
            qual = fh.readline().rstrip()  # base quality line
            if len(seq) == 0:
                break
            sequences.append(seq)
            qualities.append(qual)
    return sequences, qualities

sequences, qualities = readFastq('ads1_week4_reads.fq')

In [6]:
import itertools
from collections import defaultdict

def compute_all_overlaps(reads, k):
    """ Precompute all overlaps of length >= k between different reads """
    overlaps = {}
    for a, b in itertools.permutations(reads, 2):
        olen = overlap(a, b, min_length=k)
        if olen > 0:
            overlaps[(a, b)] = olen
    return overlaps

def greedy_scs_precomputed(reads, k):
    reads = reads.copy()
    overlaps = compute_all_overlaps(reads, k)

    while True:
        if not overlaps:
            break

        # Find pair with max overlap
        (read_a, read_b), olen = max(overlaps.items(), key=lambda x: x[1])
        reads.remove(read_a)
        reads.remove(read_b)
        merged = read_a + read_b[olen:]
        reads.append(merged)

        # Recompute overlaps involving new merged read
        overlaps = {
            (a, b): o for (a, b), o in overlaps.items()
            if a in reads and b in reads and a != b
        }
        for r in reads:
            if r != merged:
                o1 = overlap(r, merged, k)
                o2 = overlap(merged, r, k)
                if o1 > 0:
                    overlaps[(r, merged)] = o1
                if o2 > 0:
                    overlaps[(merged, r)] = o2

    return ''.join(reads)


In [7]:
%%time
for k in range(30, 1, -1):  # 30 is enough, no need to start at 100
    genome = greedy_scs_precomputed(sequences, k)
    if len(genome) == 15894:
        print("Genome assembled!")
        print("A count:", genome.count('A'))
        print("T count:", genome.count('T'))
        break


Genome assembled!
A count: 4633
T count: 3723
CPU times: total: 2min 35s
Wall time: 2min 36s


#### Question 4

How many Ts are there in the full, assembled genome from the previous question?